In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import joblib
import tensorflow as tf
from carla.recourse_methods import GrowingSpheres, Dice, ActionableRecourse, CEM
from carla import MLModelCatalog

c:\ProgramData\Anaconda3\envs\carla\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
# Read data
df = pd.read_csv('../data/adult_cleaned.csv')
X = df[set(df.columns) - set(['income'])]
Y = df['income']

df.columns

Index(['age', 'capital.loss', 'education.num', 'capital.gain',
       'hours.per.week', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Unknown_workclass',
       'Without-pay', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct',
       'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv',
       'Sales', 'Tech-support', 'Transport-moving', 'Unknown_occupation',
       'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White',
       'Female', 'Male', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba',
       'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France',
       'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands',
       'Honduras', 'Hong', 'Hung

In [3]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((85,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# load artificial neural network from catalog
model.load_weights('../models/adult_NN.h5')
model.summary()

[WARNING] From C:\Users\ignacy\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor [deprecation.py new_func]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                5504      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)               

In [4]:
print(Y.iloc[0:5])
model.predict(X.iloc[0:5])

0    <=50K
1    <=50K
2    <=50K
3    <=50K
4    <=50K
Name: income, dtype: object


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [6]:
from carla.data.catalog import CsvCatalog

continuous = ['age', 'capital.loss', 'education.num', 'capital.gain', 'hours.per.week']
categorical = ['Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Unknown_workclass',
       'Without-pay', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct',
       'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv',
       'Sales', 'Tech-support', 'Transport-moving', 'Unknown_occupation',
       'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White',
       'Female', 'Male', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba',
       'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France',
       'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands',
       'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy',
       'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua',
       'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland',
       'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand',
       'Trinadad&Tobago', 'United-States', 'Unknown_country', 'Vietnam',
       'Yugoslavia']
immutable = ["Female", "Male", 'age']

dataset = CsvCatalog(file_path="../data/adult_cleaned.csv",
                     continuous=continuous,
                     categorical=categorical,
                     immutables=immutable,
                     target='income')

dataset.df_train.columns

Index(['age', 'capital.loss', 'education.num', 'capital.gain',
       'hours.per.week', 'income', 'Federal-gov_1', 'Local-gov_1',
       'Never-worked_1', 'Private_1', 'Self-emp-inc_1', 'Self-emp-not-inc_1',
       'State-gov_1', 'Unknown_workclass_1', 'Without-pay_1', 'Divorced_1',
       'Married-AF-spouse_1', 'Married-civ-spouse_1',
       'Married-spouse-absent_1', 'Never-married_1', 'Separated_1',
       'Widowed_1', 'Adm-clerical_1', 'Armed-Forces_1', 'Craft-repair_1',
       'Exec-managerial_1', 'Farming-fishing_1', 'Handlers-cleaners_1',
       'Machine-op-inspct_1', 'Other-service_1', 'Priv-house-serv_1',
       'Prof-specialty_1', 'Protective-serv_1', 'Sales_1', 'Tech-support_1',
       'Transport-moving_1', 'Unknown_occupation_1', 'Amer-Indian-Eskimo_1',
       'Asian-Pac-Islander_1', 'Black_1', 'Other_1', 'White_1', 'Female_1',
       'Male_1', 'Cambodia_1', 'Canada_1', 'China_1', 'Columbia_1', 'Cuba_1',
       'Dominican-Republic_1', 'Ecuador_1', 'El-Salvador_1', 'England_

In [ ]:
continuous = ["age", "fnlwgt", "education.num", "capital.gain","hours.per.week", "capital.loss"]
categorical = ["marital.status", "native.country", "occupation", "race", "relationship", "sex", "workclass"]
immutable = ["age", "sex"]

dataset = CsvCatalog(file_path="../data/adult.csv",
                     continuous=continuous,
                     categorical=categorical,
                     immutables=immutable,
                     target='income')

In [ ]:
from models.tfmodel import TFModelAdult
model = TFModelAdult(dataset)

In [ ]:
hyperparams = {}
# load a recourse model and pass black box model
gs = Dice(model, hyperparams)

# generate counterfactual examples
counterfactuals = gs.get_counterfactuals(X.iloc[0:5])
counterfactuals

In [ ]:
dataset.df_test.columns